In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Throwing exceptions

> An *exception* is an event, which occurs during the execution of a program, that disrupts the normal flow of the program's instructions. 
>
> When an error occurs within a method [or constructor], the method [or constructor] creates an object and hands it off to the runtime system. The object, called an exception object, contains information about the error, including its type and the state of the program when the error occurred. Creating an exception object and handing it to the runtime system is called *throwing an exception*.

[From Oracle's *Exceptions* tutorial](https://docs.oracle.com/javase/tutorial/essential/exceptions/definition.html)

## Creating `Domino` and `Card` objects

The `Domino` class should allow users to create dominoes having specific values; for example, a user might want to create a domino with values 3 and 4. Similar, for the `Card` class a user might want to create the card '8 of hearts'.

The problem that we must address is what happens if a user tries to create a domino with a value outside of the range 0 to 6 or tries to create the card '0 of pasta'. In other words, we have to address the problem that occurs when a user tries to create an object using values that do not satisfy the class invariants.

Recall that a class invariant is a condition that an object of a class must ensure is true immediately after the object is first created and after every public method is invoked using the object. The `Domino` class has two invariants:

```java
val1 >= Domino.MIN_VALUE && val1 <= Domino.MAX_VALUE
val2 >= Domino.MIN_VALUE && val2 <= Domino.MAX_VALUE
```

In other words, both values of the domino must always be between 0 and 6. If a user tries to create a domino with values outside of the range 0 through 6 then the `Domino` class cannot simultaneously satisfy the user's request and the class invariants. 

The `Card` class has two invariants:

```java
rank is equal to one of the elements in Card.RANKS
suit is equal to one of the elements in Card.SUITS
```

If a user tries to create the card '0 of pasta' then the `Card` class cannot simultaneously satisfy the user's request and the class invariants.

A user should not be able to create an object with an invalid state because doing so prevents the class from ensuring that its invariants are true. The standard Java way of dealing with such situations is to test the values that the user is using to initialize the state of the object and throw an exception if the values are not suitable.

We have already seen several examples where a method throws an exception. For example, an array will throw an `ArrayIndexOutOfBoundsException` if the programmer tries to access an element using an invalid index:

In [ ]:
int[] arr = new int[1];
arr[5] = 1;

Another example is that a string will throw a `NullPointerException` if we try to replace all occurrences of the `null` string with the string `"x"`:

In [ ]:
String s = "abc";
s.replace(null, "x");

## How to throw an exception

To throw an exception the programmer creates an exception object and the uses the keyword `throw` to throw the exception. For the time being we are interested in the kinds of exceptions that indicate incorrect usage of a method or constructor. Such kinds of exceptions are called *runtime exceptions* or *unchecked exceptions*. The name *runtime exception* is misleading because it suggests that there are also compile-time exceptions; this is not the case. Exceptions can only occur when a program is run.

There are many different types of runtime exceptions in the Java standard library that the programmer can throw; furthermore, the programmer can create their own exception classes to throw. A list of the standard library runtime exceptions can be found under the section *Direct Known Subclasses* in the documentation for the [RuntimeException](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/RuntimeException.html) class. A very short list of commonly thrown runtime exceptions is shown below:

- [`IllegalArgumentException`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/IllegalArgumentException.html): thrown to indicate that a method has been passed an illegal or inappropriate argument.
- [`IndexOutOfBoundsException`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/IndexOutOfBoundsException.html): thrown to indicate that an index of some sort (such as to an array, to a string, or to a vector) is out of range.
- [`NullPointerException`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/NullPointerException.html): thrown when an application attempts to use `null` in a case where an object is required.

Throwing an exception is very easy in Java. Simply write `throw` followed by a reference to an exception object; usually, the exception object is created on the same line as the throw statement. For example, suppose that we detect that a value that we want to take the square root of is negative; we can throw an exception to indicate that an unusual result will occur if we compute the square root of the value:

In [ ]:
double value = -1.0;    // change value to some non-negative value to see what happens

if (value < 0.0) {
    throw new IllegalArgumentException("cannot compute square root of a negative value: " + value);
}
double rt = Math.sqrt(value);
System.out.println(rt);

The constructor of exception classes accept a string that can contain useful information for the programmer to trace the cause of the exception. In the example above, we indicate why the exception was thrown and what the problematic value was.

## What happens when an exception is thrown

The `throw` statement causes whatever method was running to immediately stop running. The runtime system of the JVM then attempts to find something that can handle the exception.

> The set of possible "somethings" to handle the exception is the ordered list of methods that had been called to get to the method where the error occurred. The list of methods is known as the call stack (see the next figure).

![Call stack.](../resources/images/exceptions-callstack.gif)

The runtime system searches the methods in the call stack from top to bottom looking for a block of code called an *exception handler*. If an exception handler is found that can handle the type of exception that was thrown then program flow resumes in the exception handler. If no suitable exception handler is found then the program terminates (as in the square root example above).

## Checked versus unchecked exceptions

As mentioned above, runtime exceptions are also called unchecked exceptions. Methods and constructors that might throw unchecked exceptions can be used by the programmer without having to take any special steps. If the programmer takes care to ensure that methods are called with suitable argumements then no runtime exceptions should ever be thrown during the lifetime of a running program (unless something happens to the physical computing environment such as failure of a memory stick or a CPU error).

Not surprisingly there are also *checked exceptions*. When using a method or constructor that can throw a checked exception the programmer has two choices:

1. modify the header of the method or constructor that the programmer is currently writing so that the header specifies that the method may throw an unchecked exception (see [Specifying the Exceptions Thrown by a Method](https://docs.oracle.com/javase/tutorial/essential/exceptions/declaring.html) for details).
2. implement an exception handler inside of the method or constuctor that the programmer is currently implementing

## What is an exception handler

An exception handler is a group of blocks that can handle specific types of exceptions thrown by methods that are higher in the call stack. There are three parts to an exception handler:

1. the `try` block is the block of code where a method or constructor is called that may throw a checked exception
2. the `catch` block or blocks are the blocks of code that can catch and handle a thrown exception
3. the `finally` block is a block of code that always executes when the `try` block exits, even if an unexpected exception was thrown. A `finally` block is not always present in an exception handler.

An example of using an exception handler is shown in the following cell. The code in the following cell uses a [`FileReader`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/io/FileReader.html) object to read a plain-text file one character at a time. Initializing a `FileReader` object can throw the checked exception `FileNotFoundException` and the `read` method can throw the checked exception `IOException`. The constructor call and the `read` method calls are placed inside of a `try` block. Immediately after the `try` block are two `catch` blocks that can catch and handle the `FileNotFoundException` that can be thrown by the constructor and the `IOException` that can be thrown by the `read` method. After the `catch` blocks is the `finally` block that closes the `FileReader` object.

In [2]:
import java.io.FileReader;
import java.io.FileNotFoundException;
import java.io.IOException;

String filename = "C:/Users/burton/Documents/CISC124/notes/resources/data/HelloWorld.txt";
FileReader r = null;
try {
    r = new FileReader(filename);    // may throw FileNotFoundException
    StringBuilder b = new StringBuilder();
    int c = r.read();                // may throw IOException
    while (c != -1) {
        b.append((char) c);
        c = r.read();                // may throw IOException
    }
    System.out.println(b.toString());
}
catch (FileNotFoundException x) {
    System.out.println(filename + " not found.");
}
catch (IOException x) {
    System.out.println(x);
}
finally {
    if (r != null) {
        r.close();
    }
}

public class HelloWorld {
    public static void main(String[] args) {
        System.out.println("Hello, world!");
    }
}


null

## Exercises

We are primarily interested in throwing exceptions for validating preconditions on constructors and methods. The remaining notebooks contain examples and exercises involving throwing exceptions.